<div align="center">  <img src="http://image-bed-zodiac.oss-cn-qingdao.aliyuncs.com/2018101204.png" width="1024"/> </div><br>

>本文简单介绍如何在VPS端下载YouTube视频，并通过http服务将文件从VPS下载到本地。

之前的博文使用youtube-dl下载YouTube等网站视频简单介绍了使用youtube-dl下载youtube视频的方法，同时介绍了一些速度较慢时的优化方法。但是由于天朝的网络环境实在糟糕，通常在本地下载视频速度都很不理想，就算尝试优化很多时候也只是治标不治本。不如从源头解决问题，直接在VPS端下载视频，之后再通过http或类似方式回传本地。

实际上，还有很多下载YouTube视频的方法，比如安装Chrome插件，以及通过Chrome的tampermonkey插件安装额外的网页脚本等方法。不过这些方法都不在本文讨论范围内，本文主要关注使用youtube-dl下载视频的方法。

下面就重点介绍利用caddy搭建简单http服务器实现文件回传的方法。

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#下载视频" data-toc-modified-id="下载视频-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>下载视频</a></span></li><li><span><a href="#搭建caddy服务" data-toc-modified-id="搭建caddy服务-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>搭建caddy服务</a></span><ul class="toc-item"><li><span><a href="#一并安装caddy和filemanager" data-toc-modified-id="一并安装caddy和filemanager-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>一并安装caddy和filemanager</a></span></li><li><span><a href="#新建文件夹作为网络服务器的根目录" data-toc-modified-id="新建文件夹作为网络服务器的根目录-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>新建文件夹作为网络服务器的根目录</a></span></li><li><span><a href="#写入caddy配置文件" data-toc-modified-id="写入caddy配置文件-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>写入caddy配置文件</a></span></li><li><span><a href="#开启服务" data-toc-modified-id="开启服务-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>开启服务</a></span></li><li><span><a href="#caddy-命令" data-toc-modified-id="caddy-命令-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>caddy 命令</a></span></li></ul></li><li><span><a href="#已知问题" data-toc-modified-id="已知问题-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>已知问题</a></span><ul class="toc-item"><li><span><a href="#显示启动成功，但本地无法访问" data-toc-modified-id="显示启动成功，但本地无法访问-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>显示启动成功，但本地无法访问</a></span></li></ul></li></ul></div>

## 下载视频

和之前博文[使用youtube-dl下载YouTube等网站视频](https://zodiac911.github.io/youtube-dl.html)中介绍的方法一样，通过youtube-dl下载

可能需要先在VPS上安装 youtube-dl ffmpeg 和 aria2等软件包

## 搭建caddy服务


### 一并安装caddy和filemanager

使用[doubi](https://github.com/ToyoDAdoubi/)的脚本安装

```shell
wget -N --no-check-certificate https://raw.githubusercontent.com/ToyoDAdoubi/doubi/master/caddy_install.sh 
chmod +x caddy_install.sh 
bash caddy_install.sh install http.filemanager
```

filemanager是一款用于改进caddy默认页面过于简陋的插件，允许用户通过网页下载，上传，删除，修改文件，而caddy自身是没有实现这些功能的


### 新建文件夹作为网络服务器的根目录

```shell
cd /
mkdir /usr/local/caddy/content
```

### 写入caddy配置文件

filemanager 配置方式见[filemanager GitHub](https://github.com/bronzehedwick/caddy-filemanager)

```shell
echo ":80 {
 root /usr/local/caddy/content
 timeouts none
 filemanager {
  show          /usr/local/caddy/content
  on            /
}
 gzip
 browse
}" > /usr/local/caddy/Caddyfile
```

整体复制粘贴到终端执行

### 开启服务

```shell
service caddy start
```

**现在可以通过 `http://[IP]`访问服务器了，IP为VPS分配给你的IP** 默认用户名和密码都是 admin

登录之后的页面大致是这个样子

<div align="center">  <img src="http://image-bed-zodiac.oss-cn-qingdao.aliyuncs.com/2018101206.png" width="900"/> </div><br>

之后就可以直接从VPS取回已经在VPS端下载好的文件了，就和平时从网页上下载文件一样。

### caddy 命令

```shell
service caddy start
service caddy stop
service caddy restart
service caddy status
```

## 已知问题

### 显示启动成功，但本地无法访问

这个可能是防火墙的问题，开放防火墙端口即可

```
iptables-I INPUT-m state--state NEW-m tcp-p tcp--dport端口-j ACCEPT
iptables-I INPUT-m state--state NEW-m udp-p udp--dport端口-j ACCEPT
```

或者删除防火墙规则，内容一样把 -I 换成 -D 就行了

```
iptables-D INPUT-m state--state NEW-m tcp-p tcp--dport端口-j ACCEPT
iptables-D INPUT-m state--state NEW-m udp-p udp--dport端口-j ACCEPT
```

更多问题解决方法参见 https://doub.io/jzzy-3/